In [1]:
from pyscf import gto, scf
import scipy
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 

# ancora il modo migliore per mettere cariche frazionali sulle molecole, se vai a toccare mol.charges si incazza 
#  quando deve calcolare l' initial guess 
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords()/angstrom, deltaZ)
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float)
            for i in range (len(q)):
                q[i] += deltaZ[i] 
            return self.mol.energy_nuc(q)
    return(NoSelfQMMM())

def eig(h, s):
    '''Solver for generalized eigenvalue problem
    .. math:: HC = SCE
    '''
    e, c = scipy.linalg.eigh(h, s)
    idx = np.argmax(abs(c.real), axis=0)
    c[:,c[idx,np.arange(len(e))].real<0] *= -1
    return e, c

In [2]:
molco=gto.Mole(atom="C 0 0 0 ; O 0 0 1",basis="ccpVTZ")
molco.build()
hf=scf.RHF(molco)
hf.scf()

converged SCF energy = -112.743889237044


-112.74388923704434

In [4]:
S=hf.get_ovlp()
F=hf.get_fock()
P=hf.make_rdm1()
C=hf.mo_coeff
O=np.diag(hf.mo_occ)
h1=hf.get_hcore()
g_ijkl=hf.mol.intor('int2e_sph')
def buildF(P):
    return h1+np.einsum('ijkl,kl->ij',g_ijkl,P)-0.5*(np.einsum('ijkl,jl->ik',g_ijkl,P))

In [5]:
%%time
hft=fc(hf,[1,-1])
dV=hft.get_hcore()-h1
C1=eig(F+dV,S)[1]
P1=C1.dot(O.dot(C1.T))
np.trace((h1+dV+np.einsum('ijkl,kl->ij',g_ijkl,P1)/2-np.einsum('ijkl,jl->ik',g_ijkl,P1)/4).dot(P1))+hft.energy_nuc()

TypeError: __init__() missing 2 required positional arguments: 'scf_method' and 'mm_mol'

In [6]:
%%time
hft=fc(hf,[1,-1])
hft.scf()


TypeError: __init__() missing 2 required positional arguments: 'scf_method' and 'mm_mol'